# Data Preprocessing

The following code reads in the data, and saves it in numpy arrays for later use

In [2]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import datetime
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Iterate through all of the files in the data folder, and extract the desired variable. In this case, that variable is the lattitude and logintude grid of that value. For example, the air temperature data is a grid of temperatures, corresponding to different locations on the globe. From each value, 16 years of data are read in and stored by year.

In [6]:
import os

folder = "C:/users/zdunkerton/documents/githubdata/hurricane/initial_data"
for dir in os.listdir(folder):
    data = []
    time = []
    for filename in os.listdir(folder+"/"+dir):
        x = Dataset(folder+"/"+dir+"/"+filename,'r').variables[filename.split('.')[0]]
        y = Dataset(folder+"/"+dir+"/"+filename,'r').variables["time"]
        if(x.shape[0]>1460):
            x =x[:1460,:,:]
            y = y[:1460]
        time.append(y)
        data.append(x)
    time = np.array(time)
    time = np.reshape(time, (23360,))
    data = np.array(data)
    time_name = "C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/time.npy"
    data_name = "./"+dir+".npy"
    np.save(time_name, time)
    #np.save(data_name, data)

Here I am saving and reloading the variables so I have them, but this isn't necessary.

In [3]:
temp = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/Temp_Surface_4xDaily.npy")

In [4]:
temp.shape

(16, 1460, 73, 144)

Read in the data on date at a time. For each data, read in the four values and stack them on top of each other. These layers all correspond to the same point in time. 

In [ ]:
year = 0
data= []
while year < temp.shape[0]:
    day = 0
    while day < temp.shape[1]:
        week = 0
        while week<28:
            temp1 = temp[year][day]
            temp1 = scaler.fit_transform(temp1)
            wind1 = wind[year][day]
            wind1 = scaler.fit_transform(wind1)
            humd1 = humd[year][day]
            humd1 = scaler.fit_transform(humd1)
            pres1 = press[year][day]
            pres1 = scaler.fit_transform(pres1)
            the_data = np.array([temp1,wind1,humd1,pres1])
            data.append(the_data)
            week+=1
        day+=1
    year+=1
data = np.array(data)
data.shape

In [29]:
np.save("Preprocessed_data.npy", data)

In [36]:
time = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/time.npy")

There are some days that have multiple hurricanes recorded. It is okay that times are only couted as having one hurricane, because the point is to see if there was a hurricane or not, not to determine size or location. 

In [17]:
df = pd.read_csv("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/atlantic.csv")
df = df.loc[df["ID"]=='AL152012']
print(df['Maximum Wind'].max())
df

45


,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
47975,AL152012,OSCAR,20121002,1200,,LO,12.4N,38.0W,25,1008,...,0,0,0,0,0,0,0,0,0,0
47976,AL152012,OSCAR,20121002,1800,,LO,13.6N,38.7W,25,1008,...,0,0,0,0,0,0,0,0,0,0
47977,AL152012,OSCAR,20121003,0,,LO,15.0N,39.6W,30,1007,...,0,0,0,0,0,0,0,0,0,0
47978,AL152012,OSCAR,20121003,600,,TD,16.3N,40.5W,30,1006,...,0,0,0,0,0,0,0,0,0,0
47979,AL152012,OSCAR,20121003,1200,,TD,17.4N,41.2W,30,1006,...,0,0,0,0,0,0,0,0,0,0
47980,AL152012,OSCAR,20121003,1800,,TS,18.2N,41.8W,35,1005,...,0,0,0,0,0,0,0,0,0,0
47981,AL152012,OSCAR,20121004,0,,TS,18.9N,42.1W,40,1002,...,0,0,0,0,0,0,0,0,0,0
47982,AL152012,OSCAR,20121004,600,,TS,19.5N,42.2W,40,1001,...,0,0,0,0,0,0,0,0,0,0
47983,AL152012,OSCAR,20121004,1200,,TS,20.2N,42.0W,45,998,...,0,0,0,0,0,0,0,0,0,0
47984,AL152012,OSCAR,20121004,1800,,TS,21.0N,41.5W,45,996,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df = pd.read_csv("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/atlantic.csv")
df = df.loc[df["Date"]>20000000]
storms = df['ID'].unique()
labels = []
for id in storms:
    storm = df.loc[df["ID"]== id]
    max_wind = storm["Maximum Wind"].max()
    if max_wind < 33:
        labels.append(0)
    elif max_wind >= 33 and max_wind < 63:
        labels.append(1)
    elif max_wind >= 63 and max_wind < 95:
        labels.append(2)
    elif max_wind >= 95:
        labels.append(3)

In [32]:
zero = 0
one = 0
two = 0
three = 0
for x in labels:
    if x == 0:
        zero +=1
    elif x ==1:
        one +=1
    elif x ==2:
        two +=1
    elif x ==3:
        three +=1
print(zero)
print(one)
print(two)
print(three)

26
126
63
54


In [45]:
df = pd.read_csv("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/atlantic.csv")
df = df.loc[df["Date"]>20000000]
ids = df['ID'].unique()
storms = []
for x in ids:
    current_storm = []
    storm = df.loc[df["ID"]==x]
    date = storm["Date"].values
    time = storm["Time"].values
    i = 0
    date1 = datetime.datetime(1800,1,1,0,0)
    hurricane_times = []
    while i<len(date):
        the_time = time[i]
        the_date = str(date[i])
        if (the_time !=0) and (the_time !=600) and (the_time !=1200) and (the_time !=1800):
            if the_time <=900:
                time[i] = 600
            elif (the_time >900) and (the_time<=1500):
                time[i] = 1200
            elif the_time >1500:
                time[i] = 1800
        hurricane_time = datetime.datetime(int(the_date[:4]), int(the_date[4:6]),int(the_date[6:8]),int(time[i]/100))
        converted_time = (hurricane_time-date1).total_seconds()/3600
        current_storm.append(converted_time)
        i+=1
    while i <100:
        current_storm.append(current_storm[0]-6)
        current_storm = sorted(current_storm)
        i+=1
    storms.append(current_storm)
    

In [47]:
storms = np.array(storms)


In [48]:
storms.shape

(269, 100)

In [17]:
df = pd.read_csv("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/atlantic_hurricane.csv")
i = 0
date1 = datetime.datetime(1800,1,1,0,0)
hurricane_times = []
while i<len(df):
    the_time = df.iloc[i,3]
    if (the_time !=0) and (the_time !=600) and (the_time !=1200) and (the_time !=1800):
        if the_time <=900:
            df.iloc[i,3] = 600
        elif (the_time >900) and (the_time<=1500):
            df.iloc[i,3] = 1200
        elif the_time >1500:
            df.iloc[i,3] = 1800
    hurricane_time = datetime.datetime(int(str(df.iloc[i,2])[:4]), int(str(df.iloc[i,2])[4:6]),int(str(df.iloc[i,2])[6:8]),int(df.iloc[i,3]/100))
    converted_time = (hurricane_time-date1).total_seconds()/3600
    hurricane_times.append(converted_time)
    i+=1
    
df = pd.read_csv("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/pacific_hurricane.csv")
i = 0
date1 = datetime.datetime(1800,1,1,0,0)
while i<len(df):
    the_time = df.iloc[i,3]
    if (the_time !=0) and (the_time !=600) and (the_time !=1200) and (the_time !=1800):
        if the_time <=900:
            df.iloc[i,3] = 600
        elif (the_time >900) and (the_time<=1500):
            df.iloc[i,3] = 1200
        elif the_time >1500:
            df.iloc[i,3] = 1800
    hurricane_time = datetime.datetime(int(str(df.iloc[i,2])[:4]), int(str(df.iloc[i,2])[4:6]),int(str(df.iloc[i,2])[6:8]),int(df.iloc[i,3]/100))
    converted_time = (hurricane_time-date1).total_seconds()/3600
    hurricane_times.append(converted_time)
    i+=1
    
count = 0
labels = []
for t in time:
    if t in hurricane_times:
        labels.append(1)
        count+=1
    else:
        labels.append(0)

In [18]:
count = 0
for x in labels:
    if x == 1:
        count +=1
count

2692

In [19]:
np.save("labels.npy", labels)